### [787\. Cheapest Flights Within K Stops](https://leetcode.com/problems/cheapest-flights-within-k-stops/)

Difficulty: **Medium**


There are `n` cities connected by `m` flights. Each fight starts from city `u` and arrives at `v` with a price `w`.

Now given all the cities and flights, together with starting city `src` and the destination `dst`, your task is to find the cheapest price from `src` to `dst` with up to `k` stops. If there is no such route, output `-1`.

```
Example 1:
Input: 
n = 3, edges = [[0,1,100],[1,2,100],[0,2,500]]
src = 0, dst = 2, k = 1
Output: 200
Explanation: 
The graph looks like this:

The cheapest price from city 0 to city 2 with at most 1 stop costs 200, as marked red in the picture.
```

```
Example 2:
Input: 
n = 3, edges = [[0,1,100],[1,2,100],[0,2,500]]
src = 0, dst = 2, k = 0
Output: 500
Explanation: 
The graph looks like this:

The cheapest price from city 0 to city 2 with at most 0 stop costs 500, as marked blue in the picture.
```

**Note:**

*   The number of nodes `n` will be in range `[1, 100]`, with nodes labeled from `0` to `n` `- 1`.
*   The size of `flights` will be in range `[0, n * (n - 1) / 2]`.
*   The format of each flight will be `(src,` `dst``, price)`.
*   The price of each flight will be in the range `[1, 10000]`.
*   `k` is in the range of `[0, n - 1]`.
*   There will not be any duplicated flights or self cycles.


In [66]:
# Bellman-Ford
# DP + Relaxation
# https://leetcode.com/problems/cheapest-flights-within-k-stops/discuss/340911/Understanding-Bellman-Ford
from typing import List

class Solution:
    def findCheapestPrice(self, n: int, edges: List[List[int]], src: int, dst: int, K: int) -> int:
        # dp[k][j] means distance to reach City j using at most k edges from City src
        # initially edge weight is +inifinite
        INF = float('inf')
        dp = [[INF] * (n+1) for _ in range(K+2)]
        for k in range(K+2):
            dp[k][src] = 0 # src->src weight is 0
        
        for k in range(1, K+2): # 1->K+1 => 1->N-1 in origin algorithm, which means a shortest path can have at most N−1 edges
            for edge in edges:
                u, v, w = edge[0], edge[1], edge[2]
                # relaxation: du + w < dv
                if dp[k-1][u] != INF and dp[k-1][u] + w < dp[k][v]:
                    dp[k][v] = dp[k-1][u] + w
        return -1 if dp[-1][dst] == INF else dp[-1][dst]

In [83]:
# Dijkstra
from typing import List
from collections import defaultdict
from heapq import heappush, heappop

class Solution:
    def findCheapestPrice(self, n: int, edges: List[List[int]], src: int, dst: int, K: int) -> int:
        INF = float('inf')
        graph = defaultdict(dict)
        for u, v, w in edges:
            graph[u][v] = w
        heap = [(0,src,K+1)] # cost, city, stops
        while heap:
            cost, city, stops = heappop(heap)
             # up to K stops means: up to K+1 steps including dst
            if dst == city:
                return cost
            if stops <= 0:
                continue
            #print(f"=={city}==")
            for nbr_city, nbr_cost in graph[city].items():
                new_cost = cost + nbr_cost
                new_stops = stops-1
                heappush(heap, (new_cost, nbr_city, new_stops))
            #print(heap)
        return -1

In [126]:
# Dijkstra - with better pruning
# Follow-ups: 
# 1. print all paths with cheapest-price
# 2. solve with multiple solutions
from typing import List
from collections import defaultdict
from heapq import heappush, heappop

class Solution:
    def findCheapestPrice(self, n: int, edges: List[List[int]], src: int, dst: int, K: int) -> int:
        INF = float('inf')
        graph = defaultdict(dict)
        for u, v, w in edges:
            graph[u][v] = w
        best = {} # key: (stops, city), value: cost
        paths = {}
        heap = [(0,src,K+1)] # cost, city, stops
        res = []
        final_cost = INF
        while heap:
            cost, city, stops = heappop(heap)
            #print(cost, city, stops)
             # up to K stops means: up to K+1 steps including dst
            if cost > final_cost:
                break
            if dst == city:
                final_cost = cost
                for path in paths[stops, city]:
                    res.append(path)
            if stops <= 0 or cost > best.get((stops, city), INF): # pruning by two conditions
                continue
            for nbr_city, nbr_cost in graph[city].items():
                new_cost = cost + nbr_cost
                new_stops = stops-1
                if new_cost < best.get((new_stops, nbr_city), INF):
                    heappush(heap, (new_cost, nbr_city, new_stops))
                    best[new_stops, nbr_city] = new_cost
                    paths[new_stops, nbr_city] = [ path + [nbr_city] for path in paths.get((stops, city), [[]]) ]
                elif new_cost == best.get((new_stops, nbr_city), INF):
                    paths[new_stops, nbr_city].extend([ path + [nbr_city] for path in paths.get((stops, city), [[]]) ])
            #print(heap)
        return res

In [127]:
Solution().findCheapestPrice(n = 3, edges = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, K = 1)

[[1, 2]]

In [128]:
Solution().findCheapestPrice(n = 10, edges = [[3,4,4],[2,5,6],[4,7,10],[9,6,5],[7,4,4],[6,2,10],[6,8,6],[7,9,4],[1,5,4],[1,0,4],[9,7,3],[7,0,5],[6,5,8],[1,7,6],[4,0,9],[5,9,1],[8,7,3],[1,2,6],[4,1,5],[5,2,4],[1,9,1],[7,8,10],[0,4,2],[7,2,8]], src = 6, dst = 0, K = 7)

[[8, 7, 0]]

In [129]:
# multi-path case
Solution().findCheapestPrice(n = 5, edges = [[0,1,1],[1,2,1],[0,4,1],[4,2,1],[0,2,2],[0,3,4],[2,3,2]], src = 0, dst = 3, K = 2)

[[1, 2, 3], [4, 2, 3], [2, 3], [3]]